In [9]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import sys
import nltk

#### Register path

In [11]:
dir_of_interest = "/Volumes/SECONDARY/kerja/ugm/virtualenv_maxent_ner/Program_NER/ner_maxent"
sys.path.append(dir_of_interest)

nltk.data.path.append('/Volumes/SECONDARY/applications/nltk/nltk_data')

In [12]:
import maxent as m
import func as f
import dbmodel as d

classify = m.Maxent()
func = f.Func()
dbmodel = d.DBModel()
classifier = func.open_file('/Volumes/SECONDARY/kerja/ugm/virtualenv_maxent_ner/Program_NER/ner_maxent/train.pickle')
# data = classifier.show_most_informative_features(48)
# print classifier.weights()
# print classifier.labels()

In [13]:
from IPython.display import display
from ipywidgets import widgets

#### Input Sentence Convert to NER

In [14]:
text = widgets.Text(description="Text : ", width=50)
display(text)

# 3 warga situbondo meninggal karena dbd
# air tergenang digunakan sebagai sarang nyamuk dbd

In [15]:
def handle_submit(sender):
    maxent = maximum_entropy(text.value)
    print maxent
    str_match = combination_string_matching(maxent)
    print "\n\n==================================== MaxEnt =========================================="
    print maxent["text_tweet"]
    print "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\n"
    print "================================ Combination Str Match ==============================="
    print str_match
    print "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\n"
     
def maximum_entropy(sentence):
    return classify.training_ner(sentence.encode("utf8"), classifier)

def combination_string_matching(maxent):
    sentence_ne = maxent["text_tweet"]
    print maxent["entity"]
    if "LOC" in maxent["entity"]:
        entity_location = maxent["entity"]["LOC"]
        entity_position = maxent["entity_position"]

        db_location = "indo_db"
        location_collection = "location"
        temp_location = dbmodel.is_candidate_loc(db_location, location_collection, entity_position, entity_location)
        loc_clear = dbmodel.is_real_loc(db_location, location_collection, temp_location)
        sentence_ne = dbmodel.ner_replace_loc(maxent["text_tweet"], loc_clear)
        
    if "CON" in maxent["entity"]:
        if "sakit" in maxent["entity"]["CON"]:
            sentence_ne = dbmodel.validation_CON(sentence_ne, "sakit")
            print sentence_ne
        
    return sentence_ne

In [16]:
text.on_submit(handle_submit)